<a href="https://colab.research.google.com/github/ahyder32/mgmt467-analytics-portfolio/blob/main/Lab4_PromptPlusExamples_Colab_Kaggle_GCS_BQ_DQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MGMT 467 — Prompt-Driven Lab (with Commented Examples)
## Kaggle ➜ Google Cloud Storage ➜ BigQuery ➜ Data Quality (DQ)

**How to use this notebook**
- Each section gives you a **Build Prompt** to paste into Gemini/Vertex AI (or Gemini in Colab).
- Below each prompt, you’ll see a **commented example** of what a good LLM answer might look like.
- **Do not** just uncomment and run. Use the prompt to generate your own code, then compare to the example.
- After every step, run the **Verification Prompt**, and write the **Reflection** in Markdown.

> Goal today: Download the Netflix dataset (Kaggle) → Stage on GCS → Load into BigQuery → Run DQ profiling (missingness, duplicates, outliers, anomaly flags).


### Academic integrity & LLM usage
- Use the prompts here to generate your own code cells.
- Read concept notes and write the reflection answers in your own words.
- Keep credentials out of code. Upload `kaggle.json` when asked.


## Learning objectives
1) Explain **why** we stage data in GCS and load it to BigQuery.  
2) Build an **idempotent**, auditable pipeline.  
3) Diagnose **missingness**, **duplicates**, and **outliers** and justify cleaning choices.  
4) Connect DQ decisions to **business/ML impact**.


## 0) Environment setup — What & Why
Authenticate Colab to Google Cloud so we can use `gcloud`, GCS, and BigQuery. Set **PROJECT_ID** and **REGION** once for consistency (cost/latency).

### Build Prompt (paste to LLM)
You are my cloud TA. Generate a single **Colab code cell** that:
1) Authenticates to Google Cloud in Colab,  
2) Prompts for `PROJECT_ID` via `input()` and sets `REGION="us-central1"` (editable),  
3) Exports `GOOGLE_CLOUD_PROJECT`,  
4) Runs `gcloud config set project $GOOGLE_CLOUD_PROJECT`,  
5) Prints both values. Add 2–3 comments explaining what/why.
End with a comment: `# Done: Auth + Project/Region set`.


In [1]:
# # EXAMPLE (from LLM) — Auth + Project/Region (commented; write your own cell using the prompt)
from google.colab import auth
auth.authenticate_user()

import os
PROJECT_ID = input("Enter your GCP Project ID: ").strip()
REGION = "us-central1"  # keep consistent; change if instructed
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
print("Project:", PROJECT_ID, "| Region:", REGION)

# Set active project for gcloud/BigQuery CLI
!gcloud config set project $GOOGLE_CLOUD_PROJECT
!gcloud config get-value project
# Done: Auth + Project/Region set

Enter your GCP Project ID: mgmt-467-47888-471119
Project: mgmt-467-47888-471119 | Region: us-central1
Updated property [core/project].
mgmt-467-47888-471119


### Verification Prompt
Generate a short cell that prints the active project using `gcloud config get-value project` and echoes the `REGION` you set.


In [2]:
# Verify the active project and region
!gcloud config get-value project
import os
print("Region:", os.environ.get("REGION"))

mgmt-467-47888-471119
Region: None


**Reflection:** Why do we set `PROJECT_ID` and `REGION` at the top? What can go wrong if we don’t?

Setting the PROJECT_ID and REGION at the beginning of the notebook ensures that all subsequent commands and operations are performed within the correct Google Cloud project and geographic region.

If you don't set these values explicitly, you might encounter several issues:

1.   **Incorrect Project/Region**: Commands might default to a different project or region, leading to resources being created or accessed in the wrong place. This can cause confusion, unexpected costs, and difficulty managing your resources.
2.   **Command Failures**: Many gcloud and bq commands require a project to be set. Without it, these commands will fail.
3. **Inconsistency**: Different parts of your notebook or different sessions might use different projects or regions, making your work irreproducible and harder to debug.
4. **Cost Management**: Resources in different regions can have different costs. Setting a consistent region helps manage and predict your cloud spending.

## 1) Kaggle API — What & Why
Use Kaggle CLI for reproducible downloads. Store `kaggle.json` at `~/.kaggle/kaggle.json` with `0600` permissions to protect secrets.

### Build Prompt
Generate a **single Colab code cell** that:
- Prompts me to upload `kaggle.json`,
- Saves to `~/.kaggle/kaggle.json` with `0600` permissions,
- Prints `kaggle --version`.
Add comments about security and reproducibility.


In [3]:
# # EXAMPLE (from LLM) — Kaggle setup (commented)
from google.colab import files
print("Upload your kaggle.json (Kaggle > Account > Create New API Token)")
uploaded = files.upload()
# #
import os
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded[list(uploaded.keys())[0]])
    os.chmod('/root/.kaggle/kaggle.json', 0o600)  # owner-only

!kaggle --version

Upload your kaggle.json (Kaggle > Account > Create New API Token)


Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


### Verification Prompt
Generate a one-liner that runs `kaggle --help | head -n 20` to show the CLI is ready.


In [4]:
# Verify the Kaggle CLI is ready
!kaggle --help | head -n 20

usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         Print the Kaggle API version
  -W, --no-warn         Disable out-of-date API version warning

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set,

**Reflection:** Why require strict `0600` permissions on API tokens? What risks are we avoiding?

Requiring strict 0600 permissions on API tokens means that only the owner of the file can read or write to it. This is a crucial security measure to protect your credentials and avoid several risks:

1. **Unauthorized Access**: Prevents other users processes on the system from reading your API key. If your key is exposed, someone could potentially use it to access your accounts and resources. or
2. **Credential Theft**: Reduces the risk of your API token being stolen or misused, which could lead to unauthorized API calls, data breaches, or financial costs.
3. **Accidental Modification/Deletion**: Protects the file from being accidentally modified or deleted by other processes or users.

## 2) Download & unzip dataset — What & Why
Keep raw files under `/content/data/raw` for predictable paths and auditing.
**Dataset:** `sayeeduddin/netflix-2025user-behavior-dataset-210k-records`

### Build Prompt
Generate a **Colab code cell** that:
- Creates `/content/data/raw`,
- Downloads the dataset to `/content/data` with Kaggle CLI,
- Unzips into `/content/data/raw` (overwrite OK),
- Lists all CSVs with sizes in a neat table.
Include comments describing each step.


In [5]:
# # EXAMPLE (from LLM) — Download & unzip (commented)
!mkdir -p /content/data/raw
!kaggle datasets download -d sayeeduddin/netflix-2025user-behavior-dataset-210k-records -p /content/data
!unzip -o /content/data/*.zip -d /content/data/raw
# # # List CSV inventory
!ls -lh /content/data/raw/*.csv

Dataset URL: https://www.kaggle.com/datasets/sayeeduddin/netflix-2025user-behavior-dataset-210k-records
License(s): CC0-1.0
  0% 0.00/4.02M [00:00<?, ?B/s]
100% 4.02M/4.02M [00:00<00:00, 779MB/s]
Archive:  /content/data/netflix-2025user-behavior-dataset-210k-records.zip
  inflating: /content/data/raw/README.md  
  inflating: /content/data/raw/movies.csv  
  inflating: /content/data/raw/recommendation_logs.csv  
  inflating: /content/data/raw/reviews.csv  
  inflating: /content/data/raw/search_logs.csv  
  inflating: /content/data/raw/users.csv  
  inflating: /content/data/raw/watch_history.csv  
-rw-r--r-- 1 root root 114K Aug  2 19:36 /content/data/raw/movies.csv
-rw-r--r-- 1 root root 4.5M Aug  2 19:36 /content/data/raw/recommendation_logs.csv
-rw-r--r-- 1 root root 1.8M Aug  2 19:36 /content/data/raw/reviews.csv
-rw-r--r-- 1 root root 2.2M Aug  2 19:36 /content/data/raw/search_logs.csv
-rw-r--r-- 1 root root 1.6M Aug  2 19:36 /content/data/raw/users.csv
-rw-r--r-- 1 root root 8.9M A

### Verification Prompt
Generate a snippet that asserts there are exactly **six** CSV files and prints their names.


In [6]:
# Verify the number of CSV files and print their names
import glob
csv_files = glob.glob('/content/data/raw/*.csv')
print(f"Found {len(csv_files)} CSV files:")
for csv_file in csv_files:
    print(csv_file)
assert len(csv_files) == 6, f"Expected 6 CSV files, but found {len(csv_files)}"

Found 6 CSV files:
/content/data/raw/movies.csv
/content/data/raw/watch_history.csv
/content/data/raw/reviews.csv
/content/data/raw/search_logs.csv
/content/data/raw/users.csv
/content/data/raw/recommendation_logs.csv


**Reflection:** Why is keeping a clean file inventory (names, sizes) useful downstream?

Keeping a clean file inventory with names and sizes is useful downstream for several reasons:


1. **Auditing and Reproducibility**: It provides a clear record of the raw data files used in the analysis. This is essential for auditing purposes and ensures that others can reproduce your work exactly.
2. **Data Validation**: You can quickly verify that you have downloaded all the expected files and that their sizes are consistent with expectations. This helps catch potential issues early in the process.
3. **Troubleshooting**: If there are errors or inconsistencies later in the pipeline, you can refer back to the file inventory to rule out issues with the initial data download or extraction.
4. **Documentation**: It serves as simple documentation for the raw data assets, making it easier for others (or yourself in the future) to understand what data is available and where it is located.
5. **Resource Management**: Knowing the sizes of the files helps in planning for storage requirements and estimating processing times.

## 3) Create GCS bucket & upload — What & Why
Stage in GCS → consistent, versionable source for BigQuery loads. Bucket names must be **globally unique**.

### Build Prompt
Generate a **Colab code cell** that:
- Creates a unique bucket in `${REGION}` (random suffix),
- Saves name to `BUCKET_NAME` env var,
- Uploads all CSVs to `gs://$BUCKET_NAME/netflix/`,
- Prints the bucket name and explains staging benefits.


In [7]:
# # EXAMPLE (from LLM) — GCS staging (commented)
import uuid, os
bucket_name = f"mgmt467-netflix-{uuid.uuid4().hex[:8]}"
os.environ["BUCKET_NAME"] = bucket_name
!gcloud storage buckets create gs://$BUCKET_NAME --location="us-central1"
!gcloud storage cp /content/data/raw/* gs://$BUCKET_NAME/netflix/
print("Bucket:", bucket_name)
# # # Verify contents
!gcloud storage ls gs://$BUCKET_NAME/netflix/

Creating gs://mgmt467-netflix-bd0c8371/...
Copying file:///content/data/raw/movies.csv to gs://mgmt467-netflix-bd0c8371/netflix/movies.csv
Copying file:///content/data/raw/README.md to gs://mgmt467-netflix-bd0c8371/netflix/README.md
Copying file:///content/data/raw/recommendation_logs.csv to gs://mgmt467-netflix-bd0c8371/netflix/recommendation_logs.csv
Copying file:///content/data/raw/reviews.csv to gs://mgmt467-netflix-bd0c8371/netflix/reviews.csv
Copying file:///content/data/raw/search_logs.csv to gs://mgmt467-netflix-bd0c8371/netflix/search_logs.csv
Copying file:///content/data/raw/users.csv to gs://mgmt467-netflix-bd0c8371/netflix/users.csv
Copying file:///content/data/raw/watch_history.csv to gs://mgmt467-netflix-bd0c8371/netflix/watch_history.csv

Average throughput: 21.0MiB/s
Bucket: mgmt467-netflix-bd0c8371
gs://mgmt467-netflix-bd0c8371/netflix/README.md
gs://mgmt467-netflix-bd0c8371/netflix/movies.csv
gs://mgmt467-netflix-bd0c8371/netflix/recommendation_logs.csv
gs://mgmt467-n

### Verification Prompt
Generate a snippet that lists the `netflix/` prefix and shows object sizes.


In [8]:
# List the objects in the netflix/ prefix with sizes
import os
bucket_name = os.environ.get("BUCKET_NAME")
if bucket_name:
    !gcloud storage ls --recursive gs://$BUCKET_NAME/netflix/
else:
    print("BUCKET_NAME environment variable is not set.")

gs://mgmt467-netflix-bd0c8371/netflix/:
gs://mgmt467-netflix-bd0c8371/netflix/README.md
gs://mgmt467-netflix-bd0c8371/netflix/movies.csv
gs://mgmt467-netflix-bd0c8371/netflix/recommendation_logs.csv
gs://mgmt467-netflix-bd0c8371/netflix/reviews.csv
gs://mgmt467-netflix-bd0c8371/netflix/search_logs.csv
gs://mgmt467-netflix-bd0c8371/netflix/users.csv
gs://mgmt467-netflix-bd0c8371/netflix/watch_history.csv


**Reflection:** Name two benefits of staging in GCS vs loading directly from local Colab.

Staging data in Google Cloud Storage (GCS) before loading it into BigQuery offers several advantages compared to loading directly from your local Colab environment. Here are two key benefits:

1. **Persistence and Availability**: Data stored in GCS is persistent and remains available even after your Colab session ends. Loading directly from a local Colab environment means the data is tied to that temporary runtime and would need to be re-uploaded for future sessions or if the runtime restarts. GCS provides a durable and accessible location for your data.
2. **Scalability and Performance**: GCS is a highly scalable and performant object storage service designed for large datasets. Loading data into BigQuery from GCS is generally more efficient and scalable, especially for larger files, as it leverages Google Cloud's infrastructure for data transfer. Loading directly from a local Colab environment can be slower and less reliable for large volumes of data due to potential network limitations and the temporary nature of the Colab runtime.

## 4) BigQuery dataset & loads — What & Why
Create dataset `netflix` and load six CSVs with **autodetect** for speed (we’ll enforce schemas later).

### Build Prompt (two cells)
**Cell A:** Create (idempotently) dataset `netflix` in US multi-region; if it exists, print a friendly message.  
**Cell B:** Load tables from `gs://$BUCKET_NAME/netflix/`:
`users, movies, watch_history, recommendation_logs, search_logs, reviews`
with `--skip_leading_rows=1 --autodetect --source_format=CSV`.
Finish with row-count queries for each table.


In [ ]:
# # EXAMPLE (from LLM) — BigQuery dataset (commented)
DATASET="netflix"
# # # Attempt to create; ignore if exists
!bq --location=US mk -d --description "MGMT467 Netflix dataset" $DATASET || echo "Dataset may already exist."

In [9]:
# # EXAMPLE (from LLM) — Load tables (commented)
tables = {
   "users": "users.csv",
   "movies": "movies.csv",
   "watch_history": "watch_history.csv",
   "recommendation_logs": "recommendation_logs.csv",
   "search_logs": "search_logs.csv",
   "reviews": "reviews.csv",
 }
import os
DATASET = "netflix" # Assuming DATASET is set in a previous cell
for tbl, fname in tables.items():
   src = f"gs://{os.environ['BUCKET_NAME']}/netflix/{fname}"
   print("Loading", tbl, "from", src)
   # Corrected bq load syntax: destination_table, source_uri
   !bq load --skip_leading_rows=1 --autodetect --source_format=CSV {DATASET}.{tbl} {src}

# # # Row counts
for tbl in tables.keys():
    # Corrected bq query syntax with escaped backticks
    query = f"SELECT '{tbl}' AS table_name, COUNT(*) AS n FROM `{os.environ['GOOGLE_CLOUD_PROJECT']}.netflix.{tbl}`"
    !bq query --nouse_legacy_sql "{query}"

Loading users from gs://mgmt467-netflix-bd0c8371/netflix/users.csv
Waiting on bqjob_r3d20b9f248349e89_0000019a0c9bca9e_1 ... (1s) Current status: DONE   
Loading movies from gs://mgmt467-netflix-bd0c8371/netflix/movies.csv
Waiting on bqjob_r1a23203a940be1c2_0000019a0c9be0ff_1 ... (1s) Current status: DONE   
Loading watch_history from gs://mgmt467-netflix-bd0c8371/netflix/watch_history.csv
Waiting on bqjob_r591105dcf46ccf48_0000019a0c9bf6e6_1 ... (2s) Current status: DONE   
Loading recommendation_logs from gs://mgmt467-netflix-bd0c8371/netflix/recommendation_logs.csv
Waiting on bqjob_r1174039eedceb1a0_0000019a0c9c1130_1 ... (2s) Current status: DONE   
Loading search_logs from gs://mgmt467-netflix-bd0c8371/netflix/search_logs.csv
Waiting on bqjob_r2c7dc89d3ed1a955_0000019a0c9c2dbc_1 ... (2s) Current status: DONE   
Loading reviews from gs://mgmt467-netflix-bd0c8371/netflix/reviews.csv
Waiting on bqjob_r1874c19af455392f_0000019a0c9c4899_1 ... (1s) Current status: DONE   
/bin/bash: lin

### Verification Prompt
Generate a single query that returns `table_name, row_count` for all six tables in `${GOOGLE_CLOUD_PROJECT}.netflix`.


In [11]:
# Generate a single query for row counts across all tables
%%bigquery
SELECT 'users' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-47888-471119.netflix.users`
UNION ALL
SELECT 'movies' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-47888-471119.netflix.movies`
UNION ALL
SELECT 'watch_history' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-47888-471119.netflix.watch_history`
UNION ALL
SELECT 'recommendation_logs' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-47888-471119.netflix.recommendation_logs`
UNION ALL
SELECT 'search_logs' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-47888-471119.netflix.search_logs`
UNION ALL
SELECT 'reviews' AS table_name, COUNT(*) AS row_count FROM `mgmt-467-47888-471119.netflix.reviews`

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,movies,1040
1,users,10300
2,search_logs,26500
3,reviews,15450
4,recommendation_logs,52000
5,watch_history,105000


**Reflection:** When is `autodetect` acceptable? When should you enforce explicit schemas and why?

Autodetect is good for quick exploration on simple, consistent data. Use explicit schemas for critical data quality, large datasets, production pipelines, complex types, and performance control. Explicit schemas provide reliability and precision, while autodetect offers speed and convenience.

## 5) Data Quality (DQ) — Concepts we care about
- **Missingness** (MCAR/MAR/MNAR). Impute vs drop. Add `is_missing_*` indicators.
- **Duplicates** (exact vs near). Double-counted engagement corrupts labels & KPIs.
- **Outliers** (IQR). Winsorize/cap vs robust models. Always **flag** and explain.
- **Reproducibility**. Prefer `CREATE OR REPLACE` and deterministic keys.


### 5.1 Missingness (users) — What & Why
Measure % missing and check if missingness depends on another variable (MAR) → potential bias & instability.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Total rows and % missing in `region`, `plan_tier`, `age_band` from `users`.
2) `% plan_tier missing by region` ordered descending. Add comments on MAR.


In [12]:
%%bigquery
-- Users: Total rows and % missing per column (region, plan_tier, age_band)
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `mgmt-467-47888-471119.netflix.users`
)
SELECT n,
       ROUND(SAFE_DIVIDE(100*miss_region, n),2) AS pct_missing_region,
       ROUND(SAFE_DIVIDE(100*miss_plan, n),2)   AS pct_missing_plan_tier,
       ROUND(SAFE_DIVIDE(100*miss_age, n),2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,10300,0.0,0.0,11.93


In [13]:
%%bigquery
-- Users: % plan_tier missing by region (examining MAR)
-- If the percentage of missing values for a column varies significantly across categories
-- of another variable (like region), it might indicate Missing At Random (MAR).
SELECT state_province,
       COUNT(*) AS n,
       ROUND(SAFE_DIVIDE(100*COUNTIF(subscription_plan IS NULL), COUNT(*)),2) AS pct_missing_plan_tier
FROM `mgmt-467-47888-471119.netflix.users`
GROUP BY state_province
ORDER BY pct_missing_plan_tier DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,state_province,n,pct_missing_plan_tier
0,Alberta,310,0.0
1,Arizona,361,0.0
2,British Columbia,309,0.0
3,California,363,0.0
4,Florida,353,0.0
5,Georgia,359,0.0
6,Illinois,355,0.0
7,Indiana,374,0.0
8,Manitoba,299,0.0
9,Maryland,351,0.0


### Verification Prompt
Generate a query that prints the three missingness percentages from (1), rounded to two decimals.


In [14]:
%%bigquery
WITH base AS (
  SELECT COUNT(*) n,
         COUNTIF(state_province IS NULL) miss_region,
         COUNTIF(subscription_plan IS NULL) miss_plan,
         COUNTIF(age IS NULL) miss_age
  FROM `mgmt-467-47888-471119.netflix.users`
)
SELECT n,
       ROUND(SAFE_DIVIDE(100*miss_region, n),2) AS pct_missing_region,
       ROUND(SAFE_DIVIDE(100*miss_plan, n),2)   AS pct_missing_plan_tier,
       ROUND(SAFE_DIVIDE(100*miss_age, n),2)    AS pct_missing_age_band
FROM base;

Query is running:   0%|          |

Downloading:   0%|          |

,n,pct_missing_region,pct_missing_plan_tier,pct_missing_age_band
0,10300,0.0,0.0,11.93


**Reflection:** Which columns are most missing? Hypothesize MCAR/MAR/MNAR and why.

The most missing column is `age_band` (11.93%). `Region` and `plan_tier` have no missing values (likely MCAR). `Age_band` missingness is likely MCAR or possibly MAR, but further analysis is needed to confirm if it's related to other variables. It's less likely to be MNAR.

### 5.2 Duplicates (watch_history) — What & Why
Find exact duplicate interaction records and keep **one best** per group (deterministic policy).

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Report duplicate groups on `(user_id, movie_id, event_ts, device_type)` with counts (top 20).
2) Create table `watch_history_dedup` that keeps one row per group (prefer higher `progress_ratio`, then `minutes_watched`). Add comments.


In [15]:
%%bigquery
-- Report duplicate groups on (user_id, movie_id, event_ts, device_type) with counts (top 20)
SELECT user_id, movie_id, device_type, COUNT(*) AS dup_count
FROM `mgmt-467-47888-471119.netflix.watch_history`
GROUP BY user_id, movie_id, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,user_id,movie_id,device_type,dup_count
0,user_03310,movie_0640,Smart TV,4
1,user_00391,movie_0893,Laptop,4
2,user_07242,movie_0463,Desktop,3
3,user_02976,movie_0987,Desktop,3
4,user_05811,movie_0177,Desktop,3
5,user_06799,movie_0458,Desktop,3
6,user_07875,movie_0845,Desktop,3
7,user_02359,movie_0108,Desktop,3
8,user_06085,movie_0346,Desktop,3
9,user_08308,movie_0641,Desktop,3


In [16]:
%%bigquery
-- Create table watch_history_dedup that keeps one row per group (prefer higher progress_ratio, then minutes_watched)
-- Use a window function to assign a rank to each row within a partition of the grouping columns.
-- The ORDER BY clause in the window function defines the "keep-one" policy.
CREATE OR REPLACE TABLE `mgmt-467-47888-471119.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `mgmt-467-47888-471119.netflix.watch_history` h
)
WHERE rk = 1;

Query is running:   0%|          |

""


### Verification Prompt
Generate a before/after count query comparing raw vs `watch_history_dedup`.


In [17]:
%%bigquery
-- Compare row counts before and after deduplication
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count
FROM `mgmt-467-47888-471119.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count
FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`;

Query is running:   0%|          |

Downloading:   0%|          |

,table_name,row_count
0,watch_history_raw,105000
1,watch_history_dedup,99878


**Reflection:** Why do duplicates arise (natural vs system-generated)? How do they corrupt labels and KPIs?

Duplicates can be natural (rare, identical real-world events) or, more commonly, system-generated (from technical issues in data pipelines). They corrupt labels and KPIs by inflating counts, skewing metrics, creating incorrect ML labels, and biasing analysis, leading to inaccurate insights and wasted resources.

### 5.3 Outliers (minutes_watched) — What & Why
Estimate extreme values via IQR; report % outliers; **winsorize** to P01/P99 for robustness while also **flagging** extremes.

### Build Prompt
Generate **two BigQuery SQL cells**:
1) Compute IQR bounds for `minutes_watched` on `watch_history_dedup` and report % outliers.
2) Create `watch_history_robust` with `minutes_watched_capped` capped at P01/P99; return quantile summaries before/after.


In [18]:
%%bigquery
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `mgmt-467-47888-471119.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

Query is running:   0%|          |

Downloading:   0%|          |

,outliers,total,pct_outliers
0,3477,99878,3.48


In [19]:
%%bigquery
CREATE OR REPLACE TABLE `mgmt-467-47888-471119.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS minutes_watched_capped
FROM `mgmt-467-47888-471119.netflix.watch_history_dedup` h, q;

# # -- Quantiles before vs after
# # WITH before AS (
# #   SELECT 'before' AS which, APPROX_QUANTILES(minutes_watched, 5) AS q
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_dedup`
# # ),
# # after AS (
# #   SELECT 'after' AS which, APPROX_QUANTILES(minutes_watched_capped, 5) AS q
# #   FROM `${GOOGLE_CLOUD_PROJECT}.netflix.watch_history_robust`
# # )
# # SELECT * FROM before UNION ALL SELECT * FROM after;

Query is running:   0%|          |

""


### Verification Prompt
Generate a query that shows min/median/max before vs after capping.


In [20]:
%%bigquery
WITH before AS (
  SELECT
    'before' AS which,
    MIN(watch_duration_minutes) AS min_watched,
    APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_watched,
    MAX(watch_duration_minutes) AS max_watched
  FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`
),
after AS (
  SELECT
    'after' AS which,
    MIN(minutes_watched_capped) AS min_watched,
    APPROX_QUANTILES(minutes_watched_capped, 2)[OFFSET(1)] AS median_watched,
    MAX(minutes_watched_capped) AS max_watched
  FROM `mgmt-467-47888-471119.netflix.watch_history_robust`
)
SELECT * FROM before
UNION ALL
SELECT * FROM after;

Query is running:   0%|          |

Downloading:   0%|          |

,which,min_watched,median_watched,max_watched
0,before,0.2,50.8,799.3
1,after,4.5,51.0,204.1


**Reflection:** When might capping be harmful? Name a model type less sensitive to outliers and why.

Capping can be harmful if outliers are valid data or if the process generating them is important to understand. Tree-based models (like Decision Trees, Random Forests) are less sensitive to outliers because they use splits based on thresholds rather than being heavily influenced by extreme values like linear or distance-based models.

### 5.4 Business anomaly flags — What & Why
Human-readable flags help both product decisioning and ML features (e.g., binge behavior).

### Build Prompt
Generate **three BigQuery SQL cells** (adjust if columns differ):
1) In `watch_history_robust`, compute and summarize `flag_binge` for sessions > 8 hours.
2) In `users`, compute and summarize `flag_age_extreme` if age can be parsed from `age_band` (<10 or >100).
3) In `movies`, compute and summarize `flag_duration_anomaly` where `duration_min` < 15 or > 480 (if exists).
Each cell should output count and percentage and include 1–2 comments.


In [21]:
%%bigquery
-- Summarize flag_binge for sessions > 8 hours in watch_history_robust
-- A session is considered a binge if minutes_watched exceeds 8 hours (480 minutes).
SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_binge
FROM `mgmt-467-47888-471119.netflix.watch_history_robust`;

Query is running:   0%|          |

Downloading:   0%|          |

,sessions_over_8h,total,pct_flag_binge
0,638,99878,0.64


In [22]:
%%bigquery
-- Summarize flag_age_extreme in users
-- Flag users with extreme ages (below 10 or above 100) based on parsing age_band.
SELECT
  COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
          SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_flag_age_extreme
FROM `mgmt-467-47888-471119.netflix.users`;

Query is running:   0%|          |

Downloading:   0%|          |

,extreme_age_rows,total,pct_flag_age_extreme
0,179,10300,1.74


In [23]:
%%bigquery
-- Summarize flag_duration_anomaly in movies
-- Flag movies with unusually short or long durations (less than 15 min or more than 8 hours).
SELECT
  COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60) AS duration_anomaly_titles,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_duration_anomaly
FROM `mgmt-467-47888-471119.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,duration_anomaly_titles,total,pct_flag_duration_anomaly
0,23,1040,2.21


### Verification Prompt
Generate a single compact summary query that returns two columns per flag: `flag_name, pct_of_rows`.


In [24]:
%%bigquery
-- Single compact summary query for anomaly flags
SELECT
  'flag_binge' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-47888-471119.netflix.watch_history_robust`
UNION ALL
SELECT
  'flag_age_extreme' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-47888-471119.netflix.users`
UNION ALL
SELECT
  'flag_duration_anomaly' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-47888-471119.netflix.movies`;

Query is running:   0%|          |

Downloading:   0%|          |

,flag_name,pct_of_rows
0,flag_binge,0.64
1,flag_age_extreme,1.74
2,flag_duration_anomaly,2.21


**Reflection:** Which anomaly flag is most common? Which would you keep as a feature and why?

The most common flag is `flag_duration_anomaly` (2.21%). I would keep `flag_binge` as a feature because it directly captures a significant user behavior pattern (binge-watching) that is highly relevant for predicting engagement and other key metrics.

## 6) Save & submit — What & Why
Reproducibility: save artifacts and document decisions so others can rerun and audit.

In [25]:
# Collect all DQ SQL queries from relevant cells
dq_queries = """
-- DQ Query 5.1 (Missingness - Users: Total and % missing)
-- Cell ID: a5287b24
SELECT
  COUNT(*) AS total_rows,
  COUNTIF(region IS NULL) AS missing_region,
  ROUND(COUNTIF(region IS NULL) * 100.0 / COUNT(*), 2) AS pct_missing_region,
  COUNTIF(plan_tier IS NULL) AS missing_plan_tier,
  ROUND(COUNTIF(plan_tier IS NULL) * 100.0 / COUNT(*), 2) AS pct_missing_plan_tier,
  COUNTIF(age_band IS NULL) AS missing_age_band,
  ROUND(COUNTIF(age_band IS NULL) * 100.0 / COUNT(*), 2) AS pct_missing_age_band
FROM
  `${GOOGLE_CLOUD_PROJECT}.netflix.users`;

-- DQ Query 5.1 (Missingness - Users: % plan_tier missing by region)
-- Cell ID: 595a13ba
SELECT
  region,
  COUNT(*) AS total_rows,
  COUNTIF(plan_tier IS NULL) AS missing_plan_tier,
  ROUND(COUNTIF(plan_tier IS NULL) * 100.0 / COUNT(*), 2) AS pct_missing_plan_tier
FROM
  `${GOOGLE_CLOUD_PROJECT}.netflix.users`
GROUP BY
  region
ORDER BY
  pct_missing_plan_tier DESC;

-- DQ Query 5.2 (Duplicates - Watch History: Report duplicate groups)
-- Cell ID: rGGCsOQRXlb1
SELECT user_id, movie_id, device_type, COUNT(*) AS dup_count
FROM `mgmt-467-47888-471119.netflix.watch_history`
GROUP BY user_id, movie_id, device_type
HAVING dup_count > 1
ORDER BY dup_count DESC
LIMIT 20;

-- DQ Query 5.2 (Duplicates - Watch History: Create dedup table)
-- Cell ID: 1g-t2JCgXlb5
CREATE OR REPLACE TABLE `mgmt-467-47888-471119.netflix.watch_history_dedup` AS
SELECT * EXCEPT(rk) FROM (
  SELECT h.*,
         ROW_NUMBER() OVER (
           PARTITION BY user_id, movie_id, device_type
           ORDER BY progress_percentage DESC, watch_duration_minutes DESC
         ) AS rk
  FROM `mgmt-467-47888-471119.netflix.watch_history` h
)
WHERE rk = 1;

-- DQ Query 5.2 (Duplicates - Watch History: Before/After count)
-- Cell ID: noUDlRksoyn4
SELECT 'watch_history_raw' AS table_name, COUNT(*) AS row_count
FROM `mgmt-467-47888-471119.netflix.watch_history`
UNION ALL
SELECT 'watch_history_dedup' AS table_name, COUNT(*) AS row_count
FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`;

-- DQ Query 5.3 (Outliers - Watch History: IQR Bounds and % outliers)
-- Cell ID: p5nCiXS_Xlb5
WITH dist AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(1)] AS q1,
    APPROX_QUANTILES(watch_duration_minutes, 4)[OFFSET(3)] AS q3
  FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`
),
bounds AS (
  SELECT q1, q3, (q3-q1) AS iqr,
         q1 - 1.5*(q3-q1) AS lo,
         q3 + 1.5*(q3-q1) AS hi
  FROM dist
)
SELECT
  COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi) AS outliers,
  COUNT(*) AS total,
  ROUND(100*COUNTIF(h.watch_duration_minutes < b.lo OR h.watch_duration_minutes > b.hi)/COUNT(*),2) AS pct_outliers
FROM `mgmt-467-47888-471119.netflix.watch_history_dedup` h
CROSS JOIN bounds b;

-- DQ Query 5.3 (Outliers - Watch History: Create robust table with capping)
-- Cell ID: SD2Ouu6YXlb5
CREATE OR REPLACE TABLE `mgmt-467-47888-471119.netflix.watch_history_robust` AS
WITH q AS (
  SELECT
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(1)]  AS p01,
    APPROX_QUANTILES(watch_duration_minutes, 100)[OFFSET(98)] AS p99
  FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`
)
SELECT
  h.*,
  GREATEST(q.p01, LEAST(q.p99, h.watch_duration_minutes)) AS minutes_watched_capped
FROM `mgmt-467-47888-471119.netflix.watch_history_dedup` h, q;

-- DQ Query 5.3 (Outliers - Watch History: Min/Median/Max before vs after capping)
-- Cell ID: bZQ17560pWzj
WITH before AS (
  SELECT
    'before' AS which,
    MIN(watch_duration_minutes) AS min_watched,
    APPROX_QUANTILES(watch_duration_minutes, 2)[OFFSET(1)] AS median_watched,
    MAX(watch_duration_minutes) AS max_watched
  FROM `mgmt-467-47888-471119.netflix.watch_history_dedup`
),
after AS (
  SELECT
    'after' AS which,
    MIN(minutes_watched_capped) AS min_watched,
    APPROX_QUANTILES(minutes_watched_capped, 2)[OFFSET(1)] AS median_watched,
    MAX(minutes_watched_capped) AS max_watched
  FROM `mgmt-467-47888-471119.netflix.watch_history_robust`
)
SELECT * FROM before
UNION ALL
SELECT * FROM after;

-- DQ Query 5.4 (Business Anomaly Flags - Watch History: Binge sessions)
-- Cell ID: cCcstMjjXlb6
SELECT
  COUNTIF(watch_duration_minutes > 8*60) AS sessions_over_8h,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_binge
FROM `mgmt-467-47888-471119.netflix.watch_history_robust`;

-- DQ Query 5.4 (Business Anomaly Flags - Users: Extreme age)
-- Cell ID: tppGlKeDXlb6
SELECT
  COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
          SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100) AS extreme_age_rows,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_flag_age_extreme
FROM `mgmt-467-47888-471119.netflix.users`;

-- DQ Query 5.4 (Business Anomaly Flags - Movies: Duration anomaly)
-- Cell ID: NxBN2oFDXlb6
SELECT
  COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60) AS duration_anomaly_titles,
  COUNT(*) AS total,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_flag_duration_anomaly
FROM `mgmt-467-47888-471119.netflix.movies`;

-- DQ Query 5.4 (Business Anomaly Flags - Summary)
-- Cell ID: PfNlsPWPp5DK
SELECT
  'flag_binge' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(watch_duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-47888-471119.netflix.watch_history_robust`
UNION ALL
SELECT
  'flag_age_extreme' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR
                                SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) > 100), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-47888-471119.netflix.users`
UNION ALL
SELECT
  'flag_duration_anomaly' AS flag_name,
  ROUND(SAFE_DIVIDE(100*COUNTIF(duration_minutes < 15 OR duration_minutes > 8*60), COUNT(*)),2) AS pct_of_rows
FROM `mgmt-467-47888-471119.netflix.movies`;
"""

# Define the filename
filename = "netflix_dq_queries.sql"

# Write the SQL queries to the file
with open(filename, "w") as f:
    f.write(dq_queries)

print(f"'{filename}' created successfully.")
print("You can download this file from the file browser on the left sidebar.")

'netflix_dq_queries.sql' created successfully.
You can download this file from the file browser on the left sidebar.


<>:128: SyntaxWarning: invalid escape sequence '\d'
<>:128: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2040696791.py:128: SyntaxWarning: invalid escape sequence '\d'
  COUNTIF(SAFE_CAST(REGEXP_EXTRACT(CAST(age AS STRING), r'\d+') AS INT64) < 10 OR


### Build Prompt
Generate a checklist (Markdown) students can paste at the end:
- Save this notebook to the team Drive.
- Export a `.sql` file with your DQ queries and save to repo.
- Push notebook + SQL to the **team GitHub** with a descriptive commit.
- Add a README with your `PROJECT_ID`, `REGION`, bucket, dataset, and today’s row counts.


## Grading rubric (quick)
- Profiling completeness (30)  
- Cleaning policy correctness & reproducibility (40)  
- Reflection/insight (20)  
- Hygiene (naming, verification, idempotence) (10)
